# Jacobian of Four-Bar Mechanism 

The knee joint is actuated but the ankle joint is passive.  

## Todo's

  - consider $\theta_0$

In [57]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

from sympy import symbols, sin, cos, sqrt, asin, atan


In [71]:
# Actuation (knee)
theta = symbols('theta')

# Passive joint (ankle) 
psi = symbols('psi')


In [59]:

# Constants defined by geometry.
theta0 = symbols('theta0')

shank, p1, p2, p3 = symbols('shank p1 p2 p3')



In [60]:
# Manually specified structure of equations. We first introduce variables based on the kinematic structure only.
# These can easily be precomputed in code. Here, they are just mentioned as a reference.

k1 = shank / p1
k2 = shank / p3
k3 = (shank**2 + p1**2 + p3**2 - p2**2 ) / (2 * p1 * p3)


# directly use these variables
k1, k2, k3 = symbols('k1 k2 k3')


## Relationship between ankle ($\psi$) and knee ($\theta$)

In [61]:

# some helpers
cT = cos(theta)
sT = sin(theta)


A = k1 * cT + k2 + k3 + cT # C.34
B = -2*sT # C.35
C = k1 * cT - k2 + k3 - cT # C.36

# ankle
psi = 2 * atan((-B + sqrt(B * B - 4 * A * C)) / (2 * A)) #  C.39

psi = sp.simplify(psi)

psi_of_theta = psi



In [62]:
psi

      ⎛   ______________________________________________________________      
      ⎜  ╱     2    2                         2                   2           
      ⎜╲╱  - k₁ ⋅cos (θ) - 2⋅k₁⋅k₃⋅cos(θ) + k₂  + 2⋅k₂⋅cos(θ) - k₃  + 1  + sin
2⋅atan⎜───────────────────────────────────────────────────────────────────────
      ⎝                       k₁⋅cos(θ) + k₂ + k₃ + cos(θ)                    

   ⎞
   ⎟
(θ)⎟
───⎟
   ⎠

### Derivatives

In [63]:
from sympy import diff

dpsi_dtheta = diff(psi, theta)

dpsi_dtheta

  ⎛                                                                           
  ⎜                                                                           
  ⎜                                                                           
  ⎜                     ⎛   __________________________________________________
  ⎜                     ⎜  ╱     2    2                         2             
  ⎜(k₁⋅sin(θ) + sin(θ))⋅⎝╲╱  - k₁ ⋅cos (θ) - 2⋅k₁⋅k₃⋅cos(θ) + k₂  + 2⋅k₂⋅cos(θ
2⋅⎜───────────────────────────────────────────────────────────────────────────
  ⎜                                                               2           
  ⎝                                 (k₁⋅cos(θ) + k₂ + k₃ + cos(θ))            
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                ⎛   __________________________
                                                ⎜  ╱

In [64]:
# We can implement the term dpsi_dtheta now in code. Therefore, we assume from now on that it is given.

from sympy import Function
psi = Function('psi')(theta)

## Coordinate systems

In [65]:

from sympy.vector import CoordSys3D, AxisOrienter
from sympy import Matrix

# The reference system at the origin of the ankle. It is defined as follows:
# -z is the rotation axis of the joint
# x points upwards (away from the platform, aligned with the global z axis)
B = CoordSys3D('base')

orienter_psi = AxisOrienter(psi, -B.k )

# coordinate system of the ankle joint (after applying the joint rotation)
T_ankle = B.orient_new('ankle', (orienter_psi, ))

# helper coordinate system at the origin of the knee after applying the knee rotation
T_knee_base = T_ankle.locate_new('knee_base', shank * T_ankle.i )

T_knee_base



knee_base

In [66]:

orienter_theta = AxisOrienter(theta, B.k )

# coordinate system of the knee after applying the rotation
T_knee = T_knee_base.orient_new('knee', (orienter_theta, ))

T_knee

knee

In [67]:
# express the end effector in the knee joint coordinate system (T_knee). This captures the forward kinematics of child joints.
x, y, z, alpha, beta, gamma = symbols("x y z alpha beta gamma")

from sympy.vector import BodyOrienter, Point
euler = BodyOrienter(alpha, beta, gamma, rot_order="XYZ")

P_eef = T_knee.orient_new("EEF", euler, location=(T_knee.i * x + T_knee.j * y + T_knee.k * z))



In [68]:
pos_vect = P_eef.origin.position_wrt(B)

# express the EEF in the base coordinate system (ankle base)
p_eef = pos_vect.to_matrix(B)

# o = P_eef.rotation_matrix(B)

# Jacobian

Pose of the end-effector
$$
p_{eef} = (x,y,z,\alpha, \beta, \gamma )^T
$$

The Jacobian $J \in \mathrm{R}^{6x1}$
$$
J = \frac{\partial p_{eef}}{\partial \theta}
$$


In [69]:
from sympy import diff

# position part of the Jacobian
dp_dtheta = diff(p_eef, theta)
dp_dtheta = dp_dtheta.simplify()

dp_dtheta


⎡                  d            ⎛d           ⎞                   ⎛d           
⎢- shank⋅sin(ψ(θ))⋅──(ψ(θ)) + x⋅⎜──(ψ(θ)) - 1⎟⋅sin(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1
⎢                  dθ           ⎝dθ          ⎠                   ⎝dθ          
⎢                                                                             
⎢                  d            ⎛d           ⎞                   ⎛d           
⎢- shank⋅cos(ψ(θ))⋅──(ψ(θ)) - x⋅⎜──(ψ(θ)) - 1⎟⋅cos(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1
⎢                  dθ           ⎝dθ          ⎠                   ⎝dθ          
⎢                                                                             
⎣                                             0                               

⎞              ⎤
⎟⋅cos(θ - ψ(θ))⎥
⎠              ⎥
               ⎥
⎞              ⎥
⎟⋅sin(θ - ψ(θ))⎥
⎠              ⎥
               ⎥
               ⎦

the expression $\frac{\partial \mathbf{p}}{\partial \theta}$ above yields $\frac{\partial x}{\partial\theta}$ and $\frac{\partial y}{\partial\theta}$. $\frac{\partial z}{\partial\theta}$ is $0$.


To obtain the orientation part, we only need to consider the angle $\gamma$ as the joint mechanism is 2D (rotatation around z axis). In other words  $\frac{\partial \alpha}{\partial\theta} = \frac{\partial \beta}{\partial\theta} = 0$



It is

$$ 
\gamma = \gamma_{0} -\psi + \theta
$$

Partial derivative:

$$
\frac{\partial \gamma}{\partial\theta} = 1 - \frac{\partial \psi}{\partial\theta} 
$$




# Automatic code generation

It is advantageous to implement the equations above by hand. Sine and cosine terms can be computed once for improved efficiency.

*The following is just for reference*

In [70]:
from sympy import ccode #, cpp_generator

def generate_code():

  cpp_code = ccode(dp_dtheta[0], assign_to="dx_dtheta", standard="c11")

  with open("/tmp/expressions.cpp", "w") as f:
    f.write(cpp_code)